In [3]:
#testing uploading pdf
import PyPDF2
import io
def extract_pdf_content(content: bytes) -> str:
    try:
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(content))
        # pdf_reader = PyPDF2.PdfReader(content)
        text = ""
        for i, page in enumerate(pdf_reader.pages):
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print('error with file')
        raise

In [4]:
pdf_file = open("bank_statement_BofA.pdf", "rb")
print(extract_pdf_content(pdf_file))

error with file


TypeError: a bytes-like object is required, not 'str'